# Available XC functionals

A number of exchange–correlation (XC) functionals are available in VeloxChem.

We will calculate the electronic energy of helium using the Hartree–Fock (HF), second-order Møller–Plesset (MP2), and full configuration interaction (FCI) methods as well as Kohn–Sham density functional theory (DFT) using the available XC functionals.

In [ ]:
import matplotlib.pyplot as plt
import multipsi as mtp
import veloxchem as vlx

In [ ]:
molecule = vlx.Molecule.read_molecule_string("He   0.000  0.000  0.000")
basis = vlx.MolecularBasis.read(molecule, "cc-pvtz", ostream=None)

## Hartree–Fock

In [ ]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_results = scf_drv.compute(molecule, basis)

energies = {}
energies["HF"] = scf_drv.get_scf_energy()

## MP2

In [ ]:
mp2_drv = vlx.Mp2Driver()
mp2_drv.ostream.mute()
mp2_results = mp2_drv.compute(molecule, basis, scf_drv.mol_orbs)

energies["MP2"] = mp2_results["mp2_energy"] + energies["HF"]

## FCI

In [ ]:
space = mtp.OrbSpace(molecule, scf_drv.mol_orbs)
space.fci()

ci_drv = mtp.CIDriver()
ci_drv.ostream.mute()

ci_results = ci_drv.compute(molecule, basis, space)

energies["FCI"] = ci_results["energies"][0]

## DFT

In [ ]:
for xcfun in vlx.available_functionals():
    scf_drv.xcfun = xcfun
    scf_drv.compute(molecule, basis)
    
    energies[xcfun] = scf_drv.get_scf_energy()

## Plotting the energies

In [ ]:
for method in energies.keys():
    print(f"    {method:<12s}: {energies[method]:16.8f} a.u.")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

for i, method in enumerate(energies.keys()):
    ax.plot(i, energies[method], "o")

ax.set_xticks(range(len(energies)), energies.keys())
plt.xticks(rotation=90)

ax.set_ylim(-2.93, -2.82)
plt.ylabel("Electronic energy (a.u.)")

plt.grid(True)
plt.show()